In [1]:
import os

In [17]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

## Clone Githhub repository

In [3]:
%pwd

'c:\\Users\\Lenovo\\Desktop\\-Source-Code-Analysis-using-GenAI\\research'

In [4]:
!mkdir test_repo

A subdirectory or file test_repo already exists.


In [ ]:
repo_path = "test_repo/"
git_repo_path = "https://github.com/entbappy/End-to-end-Machine-Learning-Project-with-MLflow.git"

Repo.clone_from(git_repo_path,to_path=repo_path)

In [6]:
loader = GenericLoader.from_filesystem(repo_path+'/src/mlProject',
                                        glob = "**/*",
                                       suffixes=[".py"],
                                       parser = LanguageParser(language=Language.PYTHON, parser_threshold=500))

In [7]:
documents = loader.load()

In [8]:
len(documents)

21

# Chunkings(Context aver splittings)

In [9]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language = Language.PYTHON,
                                                             chunk_size = 2000,
                                                             chunk_overlap = 200)

In [10]:
texts = documents_splitter.split_documents(documents)

In [11]:
len(texts)

20

## Embedding model

In [14]:
load_dotenv()
OPEN_API_KEY = os.getenv("OPEN_API_KEY")


In [15]:

embeddings=OpenAIEmbeddings(disallowed_special=(),openai_api_key=OPEN_API_KEY)

## Knowledge Base(Vector DB)

In [19]:
vector_store= FAISS.from_documents(texts, embedding=embeddings)
vector_store.save_local("faiss_index")

## LLM Wraper

In [21]:
# llm = ChatOpenAI(model_name="gpt-4")
llm = ChatOpenAI(openai_api_key=OPEN_API_KEY)

In [22]:
# Buffer Memory
memory = ConversationSummaryMemory(llm=llm, memory_key = "chat_history", return_messages=True)

In [23]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vector_store.as_retriever(search_type="mmr", search_kwargs={"k":3}), memory=memory)

Q&A

In [24]:
question = "what is DataIngestion class"

In [25]:
result = qa(question)
print(result["answer"])

The `DataIngestion` class is a part of the ML project's components related to data ingestion. It is used to handle the ingestion of data files. The class instance is initialized with a configuration object (data_ingestion_config) and provides methods to download and extract zip files.
